In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from espn_api.football import League, Team, Player

from copy import copy
from collections import Counter
from tabulate import tabulate as table

In [65]:
year = 2020

login = pd.read_csv('login.csv')
manager, league_name, league_id, swid, espn_s2 = login.iloc[0]

league = League(league_id=league_id,
                year=year,
                swid=swid, 
                espn_s2=espn_s2)

week = 4
league.load_roster_week(week)

team = league.teams[0]
player = team.roster[0]

box_scores = league.box_scores(week)
box_score = box_scores[0]

In [30]:
# Get a dictionary of the starting roster slots and number of each for the League (Week 1 must have passed already)
starting_roster_slots = Counter([p.slot_position for p in league.box_scores(1)[0].home_lineup if p.slot_position not in ['BE', 'IR']])

In [95]:
box_scores[2].home_lineup[-7].points


In [103]:
def get_lineup(league: League, team: Team, week: int):
    ''' Return the lineup of the given team during the given week '''
    # Get the lineup for the team during the specified week
    box_scores = league.box_scores(week)
    for box_score in box_scores:
        if team == box_score.home_team:
            return = box_score.home_lineup
        elif team == box_score.away_team:
            return = box_score.away_lineup

def get_top_players(lineup: list, slot: str, n: int):
    ''' Takes a list of players and returns a list of the top n players based on points scored. '''
    # Gather players of the desired position
    eligible_players = []
    for player in lineup:
        if slot in player.eligibleSlots:
            eligible_players.append(player)
            
    return sorted(eligible_players, key=lambda x: x.points, reverse=True)[:n]


def get_best_lineup(lineup: list):
    ''' Returns the best possible lineup for team during the loaded week. '''
    # Save full roster
    saved_roster = copy(lineup)
    
    # Find Best Lineup
    best_lineup = []
    for slot in sorted(starting_roster_slots.keys(), key=len):  # Get best RB before best RB/WR/TE
        num_players = starting_roster_slots[slot]
        best_players = get_top_players(saved_roster, slot, num_players)
        best_lineup.extend(best_players)
        
        # Remove selected players from consideration for other slots
        for player in best_players:
            saved_roster.remove(player)

    return np.sum([player.points for player in best_lineup])


def get_best_trio(lineup: list):
    ''' Returns the the sum of the top QB/RB/Reciever trio for a team during the loaded week. '''
    qb = get_top_players(lineup, 'QB', 1)[0].points
    rb = get_top_players(lineup, 'RB', 1)[0].points
    wr = get_top_players(lineup, 'WR', 1)[0].points
    te = get_top_players(lineup, 'TE', 1)[0].points
    best_trio = round(qb + rb + max(wr, te), 2)
    return best_trio

def get_weekly_finish(league: League, team: Team, week: int):
    ''' Returns the rank of a team compared to the rest of the league by points for (for the loaded week) '''
    league_scores = [tm.scores[week-1] for tm in league.teams]
    league_scores = sorted(league_scores, reverse=True)
    return league_scores.index(team.scores[week-1]) + 1

def get_num_out(lineup: list):
    ''' Returns the (esimated) number of players who did not play for a team for the loaded week (excluding IR slot players). '''
    num_out = 0
    # TODO: write new code based on if player was injured
    return num_out

def avg_slot_score(lineup: list, slot: str):
    ''' 
    Returns the average score for starting players of a specified slot.
    `lineup` is either BoxScore().away_lineup or BoxScore().home_lineup (a list of BoxPlayers)
    '''
    return np.mean([player.points for player in lineup if player.slot_position == slot])

def sum_bench_points(lineup: list):
    ''' 
    Returns the total score for bench players
    `lineup` is either BoxScore().away_lineup or BoxScore().home_lineup (a list of BoxPlayers)
    '''
    return np.sum([player.points for player in lineup if player.slot_position == 'BE'])

def print_weekly_stats(league: League, team: Team, week: int):
    ''' Print the weekly stats for the team during a given week. '''

    lineup = get_lineup(league, team, week)
    stats_table = [['Week Score: ', team.scores[week-1]],
                   ['Best Possible Lineup: ', get_best_lineup(lineup)],
                   ['Opponent Score: ', team.schedule[week-1].scores[week-1]],
                   
                   ['Weekly Finish: ', get_weekly_finish(league, team, week)],
                   ['Best Trio: ', get_best_trio(lineup)],
                   ['Number of Injuries: ', get_num_out(lineup)],
                   ['Starting QB pts: ', avg_slot_score(lineup, 'QB')],
                   ['Avg. Starting RB pts: ', avg_slot_score(lineup, 'RB')],
                   ['Avg. Starting WR pts: ', avg_slot_score(lineup, 'WR')],
                   ['Starting TE pts: ', avg_slot_score(lineup, 'TE')],
                   ['Starting Flex pts: ', avg_slot_score(lineup, 'RB/WR/TE')],
                   ['Starting DST pts: ', avg_slot_score(lineup, r'D/ST')],
                   ['Starting K pts: ', avg_slot_score(lineup, 'K')],
                   ['Total Bench pts: ', sum_bench_points(lineup)]]
    
    print('\n', table(stats_table, headers = ['Week ' + str(week), ''], numalign = 'left'))
    
print_weekly_stats(league, team, week)


 Week 4
---------------------  ------
Week Score:            167.3
Best Possible Lineup:  199.66
Opponent Score:        148.42
Weekly Finish:         1
Best Trio:             85.86
Number of Injuries:    0
Starting QB pts:       20.9
Avg. Starting RB pts:  14.9
Avg. Starting WR pts:  22.2
Starting TE pts:       32.6
Starting Flex pts:     5.3
Starting DST pts:      12
Starting K pts:        17
Total Bench pts:       70.66


In [ ]:
#league.power_rankings(week)

In [110]:
def get_weekly_luck_index(league: League, team: Team, week: int):
    ''' 
    This function returns an index quantifying how 'lucky' a team was in a given week 
    
    Luck index:
        50% probability of playing a team with a lower record
        25% your play compared to previous weeks
        25% opp's play compared to previous weeks
    '''
    opp = team.schedule[week-1]
    num_teams = len(league.teams)
    
    # Luck Index based on where the team and its opponent finished compared to the rest of the league  
    rank = get_weekly_finish(league, team, week)
    opp_rank = get_weekly_finish(league, opp, week)

    if rank < opp_rank:                                # If the team won...
        luck_index = 5 * (rank - 1) / (num_teams - 2)  # Odds of this team playing a team with a higher score than it
    elif rank > opp_rank:                              # If the team lost or tied...
        luck_index = -5 * (num_teams - rank) / (num_teams - 2)    # Odds of this team playing a team with a lower score than it

    # If the team tied...
    elif rank < (num_teams / 2):                                      
        luck_index = -2.5 * (num_teams - rank - 1) / (num_teams - 2)  # They are only half as unlucky, because tying is not as bad as losing
    else:
        luck_index = 2.5 * (rank - 1) / (num_teams - 2)               # They are only half as lucky, because tying is not as good as winning


    # Update luck index based on how team played compared to normal
    team_score = team.scores[week - 1]
    team_avg = np.mean(team.scores[:week])
    team_std = np.std(team.scores[:week])
    if team_std != 0:
        # Get z-score of the team's performance
        z = (team_score - team_avg) / team_std
        
        # Noramlize the z-score so that a performance 3 std dev's away from the mean has an effect of 2 points on the luck index
        z_norm = z / (3*team_std) * 2.5
        luck_index += z_norm

    # Update luck index based on how opponent played compared to normal
    opp_score = opp.scores[week - 1]
    opp_avg = np.mean(opp.scores[:week])
    opp_std = np.std(opp.scores[:week])
    if team_std != 0:
        # Get z-score of the team's performance
        z = (opp_score - opp_avg) / opp_std
        
        # Noramlize the z-score so that a performance 3 std dev's away from the mean has an effect of 2 points on the luck index
        z_norm = z / (3*opp_std) * 2.5
        luck_index -= z_norm
    
    return luck_index / 10


def get_season_luck_indices(league: League, week: int):
    ''' This function returns an index quantifying how 'lucky' a team was all season long (up to a certain week) '''
    luck_indices = {team:0 for team in league.teams}
    for wk in range(1, week + 1):
        # Update luck_index for each team
        for team in league.teams:
            luck_indices[team] += get_weekly_luck_index(league, team, week)
    return luck_indices

In [111]:
# Get luck index
week = 12
sorted(get_season_luck_indices(league, week).items(), key=lambda x: x[1], reverse=True)

[(Team(DJ Chark doo doo doo doo), 5.0068028772958435),
 (Team(Milk and  Dalvin Cookies), 2.9790090731134082),
 (Team(Fresh Prince of Helaire), 2.0835728254697865),
 (Team(Can't Start Mike), -0.01267057171035671),
 (Team(Lockett the OG), -0.08357282546978607),
 (Team(Tannesaurus Rex), -1.0068028772958424),
 (Team(Antonio Brownies ), -2.9790090731134082),
 (Team(Ma Homie), -5.987329428289644)]

False

In [136]:
def sort_lineups_by_func(league: League, func):
    ''' Sorts league teams according to function. Values are sorted ascending. '''
        
    return sorted(league.teams, key=lambda x:func(x))

def sort_teams_by_func(league: League, func):
    ''' Sorts league teams according to function. Values are sorted ascending. '''
    return sorted(league.teams, key=lambda x:func(x))

def print_weekly_stats(league: League, week: int):
    ''' Prints weekly stat report for a league during a given week '''
    # Load box scores for specified week
    box_scores = league.box_scores(week)
    
    statsTable = [['Most Points Scored: ', sorted(league.teams, key=lambda x:x.scores[week-1], reverse=True)[0].owner],
                   ['Least Points Scored: ', sorted(league.teams, key=lambda x:x.scores[week-1])[0].owner],
                   ['Best Possible Lineup: ', sort_by_func(league, get_best_lineup)[-1].owner],
                   ['Best Trio: ', sort_by_func(league, get_best_trio)[-1].owner],
                   ['Worst Trio: ', sort_by_func(league, get_best_trio)[0].owner],
#                    ['Best Lineup Setter', sortDifference(league, week)[1].owner],
#                    ['Worst Lineup Setter', sortDifference(league, week)[last].owner],
#                    ['---------------------','----------------'],
#                    ['Best QBs: ', sortPositionScore(league, week, 0)[1].owner],
#                    ['Best RBs: ', sortPositionScore(league, week, 2)[1].owner],
#                    ['Best WRs: ', sortPositionScore(league, week, 4)[1].owner], 
#                    ['Best TEs: ', sortPositionScore(league, week, 6)[1].owner],
#                    ['Best Flex: ', sortPositionScore(league, week, 23)[1].owner],
#                    ['Best DST: ', sortPositionScore(league, week, 16)[1].owner],
#                    ['Best K: ', sortPositionScore(league, week, 17)[1].owner],
#                    ['Best Bench: ', sortBenchPoints(league, week)[1].owner],
#                    ['---------------------','----------------'],
#                    ['Worst QBs: ', sortPositionScore(league, week, 0)[last].owner],
#                    ['Worst RBs: ', sortPositionScore(league, week, 2)[last].owner],
#                    ['Worst WRs: ', sortPositionScore(league, week, 4)[last].owner], 
#                    ['Worst TEs: ', sortPositionScore(league, week, 6)[last].owner],
#                    ['Worst Flex: ', sortPositionScore(league, week, 23)[last].owner],
#                    ['Worst DST: ', sortPositionScore(league, week, 16)[last].owner],
#                    ['Worst K: ', sortPositionScore(league, week, 17)[last].owner],
#                    ['Worst Bench: ', sortBenchPoints(league, week)[last].owner]]
                   ]
    print('\n', table(statsTable, headers = ['Week ' + str(week), ''])) 
    
print_weekly_stats(league, week)

TypeError: 'Team' object is not iterable

In [119]:
a = [1, 2, 3, 4, 5, 4, 3, 2, 1]
def func(i):
    return -i
sorted(a, key=lambda x:func(x))

[5, 4, 4, 3, 3, 2, 2, 1, 1]

In [29]:
for t in league.teams:
    print('{:15}  {:.2f}'.format(t.owner, get_best_trio(t)))

NameError: name 'get_best_trio' is not defined

In [40]:
3.97 - 1.46

2.5100000000000002

In [11]:
league.box_score

AttributeError: 'League' object has no attribute 'box_score'

In [263]:
# league.box_scores(1)
a = {team:0 for team in league.teams}
a[team] += 10
a

{Team(Fresh Prince of Helaire): 10,
 Team(Lockett the OG): 0,
 Team(Can't Start Mike): 0,
 Team(Milk and  Dalvin Cookies): 0,
 Team(DJ Chark doo doo doo doo): 0,
 Team(Tannesaurus Rex): 0,
 Team(Ma Homie): 0,
 Team(Antonio Brownies ): 0}

[Player(Patriots D/ST)]